In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf

from tqdm.notebook import tqdm
from nasbench import api
from search_spaces import load_nasbench_101
from random_search import run_random_search, random_spec

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [3]:
nasbench = load_nasbench_101()

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 45 seconds


In [4]:
# Change the experiment setting here
# 600 runs, max 5000 samples, time limit set to 8e6 according to the paper

LOOPS = 60
MAX_SAMPLES = 500
MAX_TIME_BUDGET = 8e5

In [5]:
random_val_avg, random_test_avg = [], []
random_val_std, random_test_std = [], []

val_acc, test_acc = [], []
for budget in tqdm(range(0, int(MAX_TIME_BUDGET), 1600)): # 500 loops
    loop_val, loop_test = [], []

    for _ in range(LOOPS):
        time, best_val, best_test = run_random_search(nasbench, max_time_budget=budget)

        loop_val.append(np.max(best_val) * 100)
        loop_test.append(np.max(best_test) * 100)
    
    val_acc.append(np.mean(loop_val))
    test_acc.append(np.mean(loop_test))
    
    random_val_avg.append(np.max(val_acc))
    random_val_std.append(np.std(loop_val))
    
    random_test_avg.append(np.max(test_acc))
    random_test_std.append(np.std(loop_test))

  0%|          | 0/500 [00:00<?, ?it/s]

In [6]:
np.save('outputs/random_val_avg_by_time.npy', random_val_avg)
np.save('outputs/random_val_std_by_time.npy', random_val_std)
np.save('outputs/random_test_avg_by_time.npy', random_test_avg)
np.save('outputs/random_test_std_by_time.npy', random_test_std)